In [1]:
import sys
import numpy as np
import pandas as pd
import os
import cv2
import wandb
from datetime import datetime
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

sys.path.append('../')
from dataset.EyePACS_and_APTOS import Eye_APTOS
from dataset.messidor import MESSIDOR

# Which GPU to use
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [2]:
Eye_APTOS_data_dir_options = {
    'EyePACS': '/home/xiangjianhou/hc701-fed/preprocessed/eyepacs',
    'APTOS': '/home/xiangjianhou/hc701-fed/preprocessed/aptos',
}
MESSIDOR_data_dir_options = {
    'messidor2': '/home/xiangjianhou/hc701-fed/preprocessed/messidor2',
    'messidor_pairs' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_pairs',
    'messidor_Etienne' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Etienne',
    'messidor_Brest-without_dilation' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Brest-without_dilation'
}

In [3]:
Eye_APTOS_data_dir_options = {
    'EyePACS': '/home/xiangjianhou/hc701-fed/preprocessed/eyepacs',
    'APTOS': '/home/xiangjianhou/hc701-fed/preprocessed/aptos',
}


MESSIDOR_data_dir_options = {
    'messidor2': '/home/xiangjianhou/hc701-fed/preprocessed/messidor2',
    'messidor_pairs' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_pairs',
    'messidor_Etienne' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Etienne',
    'messidor_Brest-without_dilation' : '/home/xiangjianhou/hc701-fed/preprocessed/messidor/messidor_Brest-without_dilation'
}


APTOS_train = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['APTOS'], train=True, transform=None)
EyePACS_train = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['EyePACS'], train=True, transform=None)
MESSIDOR_2_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor2'], train=True, transform=None)
MESSIDOR_pairs_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_pairs'], train=True, transform=None)
MESSIDOR_Etienne_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Etienne'], train=True, transform=None)
MESSIDOR_Brest_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Brest-without_dilation'], train=True, transform=None)

Centerlized_train = ConcatDataset([APTOS_train, EyePACS_train, MESSIDOR_2_train, MESSIDOR_pairs_train, MESSIDOR_Etienne_train,MESSIDOR_Brest_train])
MESSIDOR_Centerlized_train = ConcatDataset([MESSIDOR_pairs_train, MESSIDOR_Etienne_train,MESSIDOR_Brest_train])

APTOS_test = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['APTOS'], train=False, transform=None)
EyePACS_test = Eye_APTOS(data_dir=Eye_APTOS_data_dir_options['EyePACS'], train=False, transform=None)
MESSIDOR_2_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor2'], train=False, transform=None)
MESSIDOR_pairs_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_pairs'], train=False, transform=None)
MESSIDOR_Etienne_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Etienne'], train=False, transform=None)
MESSIDOR_Brest_test = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Brest-without_dilation'], train=False, transform=None)

Centerlized_test = ConcatDataset([APTOS_test, EyePACS_test, MESSIDOR_2_test, MESSIDOR_pairs_test, MESSIDOR_Etienne_test,MESSIDOR_Brest_test])
MESSIDOR_Centerlized_test = ConcatDataset([MESSIDOR_pairs_test, MESSIDOR_Etienne_test,MESSIDOR_Brest_test])

In [4]:
train_dataset = DataLoader(Centerlized_train, batch_size=32, shuffle=True)
MESSIDOR_train_dataset = DataLoader(MESSIDOR_Centerlized_train, batch_size=32, shuffle=True)

In [5]:
from model.baseline import Baseline
model_demo = Baseline(backbone='densenet121',num_classes=5,pretrained=True)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_demo.parameters(), lr=0.001)
model_save_path = '/home/xiangjianhou/hc701-fed/checkpoint/MESSIDOR_3_hosptial_4class'

In [6]:
model_demo.train()
model_demo.to(device)
for i, (images, labels) in enumerate(train_dataset):
    images = images.to(device,torch.float32)
    labels = labels.to(device)
    outputs = model_demo(images)
    print(labels)
    print(labels.type())
    loss_value = loss(outputs, labels)
    break

tensor([0, 0, 0, 0, 4, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 0, 2, 0, 2, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 4], device='cuda:2')
torch.cuda.LongTensor


In [ ]:
# 
for epoch in range(100):
    model_demo.train()
    model_demo.to(device)
    for i, (images, labels) in enumerate(train_dataset):
        images = images.to(device,torch.float32)
        labels = labels.to(device)
        outputs = model_demo(images)
        loss_value = loss(outputs, labels)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        print('epoch: {}, batch: {}, loss: {}'.format(epoch, i, loss_value.item()))

In [ ]:
# test model accuracy and f1 score
from sklearn.metrics import f1_score
model_demo.eval()
model_demo.to(device)
y_true = []
y_pred = []
for i, (images, labels) in enumerate(MESSIDOR_Centerlized_test_loader):
    images = images.to(device,torch.float32)
    labels = labels.to(device)
    outputs = model_demo(images)
    outputs = torch.argmax(outputs, dim=1)
    y_true.append(labels.cpu().numpy())
    y_pred.append(outputs.cpu().numpy())
y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)
print('accuracy: {}'.format(np.mean(y_true == y_pred)))
print('f1 score: {}'.format(f1_score(y_true, y_pred, average='macro')))